# Redes Neurais

Esse notebook tratará da implementação, do treino e da avaliação inicial de redes neurais recorrentes (especificamente de células LSTM) na busca pela solução do seguinte problema de classificação: identificação de de viés político em textos. O tamanho desses textos é algo a ser arbitrado e até mesmo utilizado como comparação: podemos utilizar somente as sentenças que foram marcadas por conter viés, ou mesmo o texto na íntegra da notícia.

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit 

In [12]:
data = pd.read_csv('../data/final_dataset.csv')

## 1 - Separando o dataset

In [13]:
label_columns = ['Label_bias_Biased','Label_bias_No agreement', 'Label_bias_Non-biased']
labels = data[label_columns]
features = data.drop(label_columns, axis=1)

In [7]:
def train_valid_test_split(features, labels):
    """ Retorna uma lista de tuplas contendo os datasets de features e de labels para cada segmento (treino, validação, teste) """

    # Treino-val e Teste
    shuffle_train_test = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=892)
    train_val_indexes, test_indexes = next(shuffle_train_test.split(features.values, labels.values))
    train_val_df, train_val_labels = features.iloc[train_val_indexes], labels.iloc[train_val_indexes]
    test_df, test_labels = features.iloc[test_indexes], labels.iloc[test_indexes]

    # Treino e Validação
    shuffle_train_validate = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=124)
    train_indexes, validation_indexes = next(shuffle_train_validate.split(train_val_df.values,train_val_labels.values))
    train_df, train_labels = features.iloc[train_indexes], labels.iloc[train_indexes]
    validation_df, validation_labels = features.iloc[validation_indexes], labels.iloc[validation_indexes]


    return [(train_df, train_labels), (validation_df, validation_labels), (test_df, test_labels)]

In [16]:
train, validation, test = train_valid_test_split(features,labels)

train_df, train_labels = train
validation_df, validation_labels = validation
test_df, test_labels = test

print(f"Treino:{train_df.shape} ----- Validação: {validation_df.shape} ------ Teste: {test_df.shape}")

Treino:(848, 29) ----- Validação: (283, 29) ------ Teste: (283, 29)


## 2 - Estruturando arquitetura da rede neural

In [ ]:
import tensorflow_hub as hub
embed_size = 128

# parte textual
text_input_length = len(data['sentence'][1])
text_input = tf.keras.layers.Input(shape=(text_input_length,))
embedding_layer = tf.keras.layers.Embedding(int(vocab_table.size())+5000,input_dim=embed_size, output_dim=embed_size,mask_zero=True)
text_branch = tf.keras.layers.Flatten(embedding_layer)


# parte densa (features normais)
feature_input_length = len(data.columns) - 2
feature_input_layer = tf.keras.layers.Dense(10,input_shape=(32,feature_input_length))
feature_branch = tf.keras.Sequential().add(feature_input_layer)

model = tf.keras.Sequential([
    tf.keras.layers.concatenate([text_branch,feature_branch],axis=-1),
    tf.keras.layers.Dense(1, activation='softmax'),
    tf.keras.layers.GRU(128,return_sequences=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

TypeError: ignored